In [1]:
import os
import numpy as np
import pandas as pd
from featextra import featextra
# Define the path to the dataset directory
dataset_path = r'C:\Users\Diego\Documents\DTU\Spring2023\CDA\CDA_Projects\Case2\dataset\dataset'

# Define the names of the folders containing the response.csv file
response_folder_names = ['D1_1', 'D1_2', 'D1_3']

# Initialize the arrays to store the data
time_series_data = []
time =[]
labels = []
keywords = []

# Traverse the directory tree and extract the necessary information
for group_folder_name in os.listdir(dataset_path):
    if group_folder_name in response_folder_names:
        for subject_folder_name in os.listdir(os.path.join(dataset_path, group_folder_name)):
            if subject_folder_name.endswith('.csv'):
                continue
            for round_folder_name in os.listdir(os.path.join(dataset_path, group_folder_name, subject_folder_name)):
                for phase_folder_name in os.listdir(os.path.join(dataset_path, group_folder_name, subject_folder_name, round_folder_name)):
                    response_file_path = os.path.join(dataset_path, group_folder_name, subject_folder_name, round_folder_name, phase_folder_name, 'response.csv')
                    response_data = np.loadtxt(response_file_path, delimiter=',', skiprows=1, usecols=[1])
                    for signal_file_name in os.listdir(os.path.join(dataset_path, group_folder_name, subject_folder_name, round_folder_name, phase_folder_name)):
                        if (signal_file_name.endswith('.csv')) & (signal_file_name != 'response.csv'):
                            signal_file_path = os.path.join(dataset_path, group_folder_name, subject_folder_name, round_folder_name, phase_folder_name, signal_file_name)
                            signal_type = signal_file_name[:-4]  # remove the .csv extension
                            signal_data = pd.read_csv(signal_file_path, index_col=False)
                            # # if (len(time_series_data) != 0):
                            # #     signal_data = np.pad(signal_data, (0, time_series_data[-1].shape[0] - signal_data.shape[0]), 'constant')  # pad with zeros
                            time_series_data.append(signal_data[signal_type])
                            time.append(signal_data["time"])
                            keywords.append(f'{group_folder_name},{signal_type},{phase_folder_name},{subject_folder_name},{round_folder_name}')
                            labels.append(f'{group_folder_name}_{signal_type}_{phase_folder_name}_{subject_folder_name}_{round_folder_name}.dat')

# Convert the arrays to the desired format
time_series_data = np.array(time_series_data)
keywords = np.array(keywords).flatten()
labels = np.array(labels).flatten()
results = np.empty(shape=(time_series_data.shape[0], 777))
i = 0
for row, time, name in zip(time_series_data, time, labels):
    i += 1
    df = pd.DataFrame(row)
    df["time"] = time
    df["ID"] = name
    results[i,:] = featextra(df, row.name, n_jobs=1)

C:\Users\Diego\AppData\Local\Temp\ipykernel_21364\1289011824.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  time_series_data = np.array(time_series_data)
